# Генерация гороскопов

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 46.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path_model = "/content/gdrive/My Drive/Train models/gpt2_model_new.pt"

In [4]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers
from torch.optim import AdamW

device = 'cuda' if torch.cuda.is_available() else 'cpu'

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

import string

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

In [ ]:
path = 'https://raw.githubusercontent.com/dkagramanyan/astro/main/horoscopes.csv'
df = pd.read_csv(path)

In [ ]:
df.iloc[1]['text']

'Первый день нового года прекрасно подходит для начинаний, которые вы постоянно откладывали в прошлом году. Отбросьте все сомнения и действуйте. Упор может быть сделан как на повышение профессиональной квалификации, так и на реформацию личной жизни.'

In [ ]:
df

,date,sign,text
0,2004-01-01,aries,"Любые разногласия во мнениях скоро улягутся, а..."
1,2004-01-01,gemini,Первый день нового года прекрасно подходит для...
2,2004-01-01,taurus,Этот день как нельзя лучше подходит для улучше...
3,2004-01-01,cancer,"Первый день нового года, да еще понедельник бу..."
4,2004-01-01,leo,Львы не сорвутся в начале года с места в карье...
...,...,...,...
73472,2020-10-31,scorpio,"Этот день важен для репутации Скорпионов, их и..."
73473,2020-10-31,sagittarius,Стрельцам пора отдать дань прогрессу и духу вр...
73474,2020-10-31,capricorn,Сегодня Козероги могут рассчитывать как на нео...
73475,2020-10-31,aquarius,Водолеи проведут день не совсем обычно. Сюрпри...


In [ ]:
# def clean(text):
#     # text = text.lower()
#     # text = [w for word in text for w in word if w not in string.punctuation]
#     # text = ''.join([w for w in text])
#     return text

In [ ]:
# df['clean_text'] = df['text'].apply(clean)

In [ ]:
df.head(10)

,date,sign,text
0,2004-01-01,aries,"Любые разногласия во мнениях скоро улягутся, а..."
1,2004-01-01,gemini,Первый день нового года прекрасно подходит для...
2,2004-01-01,taurus,Этот день как нельзя лучше подходит для улучше...
3,2004-01-01,cancer,"Первый день нового года, да еще понедельник бу..."
4,2004-01-01,leo,Львы не сорвутся в начале года с места в карье...
5,2004-01-01,virgo,Сегодняшнее расположение звезд может усилить д...
6,2004-01-01,libra,"В этот день вы, судя по всему, будете в более ..."
7,2004-01-01,scorpio,"Этот день благоприятен для деловых встреч, зна..."
8,2004-01-01,sagittarius,Вероятны ухудшение самочувствия или проблемы с...
9,2004-01-01,capricorn,"Первый день года будет для вас днем, когда вам..."


In [ ]:
df['sign'].unique()

array(['aries', 'gemini', 'taurus', 'cancer', 'leo', 'virgo', 'libra',
       'scorpio', 'sagittarius', 'capricorn', 'aquarius', 'pisces'],
      dtype=object)

In [ ]:
df['tr_sign'] = df['sign'].apply(lambda x: 'Рак' if x == 'cancer' \
                                 else 'Скорпион' if x == 'scorpio'\
                                 else 'Лев' if x == 'leo'\
                                 else 'Близнецы' if x == 'gemini' \
                                 else 'Овен' if x == 'aries'\
                                 else 'Телец' if x == 'taurus'\
                                 else 'Дева' if x == 'virgo'\
                                 else 'Весы' if x == 'libra'\
                                 else 'Стрелец' if x == 'sagittarius'\
                                 else 'Козерог' if x == 'capricorn'\
                                 else 'Водолей' if x == 'aquarius'\
                                 else 'Рыбы')

In [ ]:
df.head(12)

,date,sign,text,tr_sign
0,2004-01-01,aries,"Любые разногласия во мнениях скоро улягутся, а...",Овен
1,2004-01-01,gemini,Первый день нового года прекрасно подходит для...,Близнецы
2,2004-01-01,taurus,Этот день как нельзя лучше подходит для улучше...,Телец
3,2004-01-01,cancer,"Первый день нового года, да еще понедельник бу...",Рак
4,2004-01-01,leo,Львы не сорвутся в начале года с места в карье...,Лев
5,2004-01-01,virgo,Сегодняшнее расположение звезд может усилить д...,Дева
6,2004-01-01,libra,"В этот день вы, судя по всему, будете в более ...",Весы
7,2004-01-01,scorpio,"Этот день благоприятен для деловых встреч, зна...",Скорпион
8,2004-01-01,sagittarius,Вероятны ухудшение самочувствия или проблемы с...,Стрелец
9,2004-01-01,capricorn,"Первый день года будет для вас днем, когда вам...",Козерог


In [ ]:
df['horoscope'] = ' ' + df['tr_sign'].astype(str) + '.' + ' ' + df['text'].astype(str)

In [ ]:
df.head(12)

,date,sign,text,tr_sign,horoscope
0,2004-01-01,aries,"Любые разногласия во мнениях скоро улягутся, а...",Овен,Овен. Любые разногласия во мнениях скоро уляг...
1,2004-01-01,gemini,Первый день нового года прекрасно подходит для...,Близнецы,Близнецы. Первый день нового года прекрасно п...
2,2004-01-01,taurus,Этот день как нельзя лучше подходит для улучше...,Телец,Телец. Этот день как нельзя лучше подходит дл...
3,2004-01-01,cancer,"Первый день нового года, да еще понедельник бу...",Рак,"Рак. Первый день нового года, да еще понедель..."
4,2004-01-01,leo,Львы не сорвутся в начале года с места в карье...,Лев,Лев. Львы не сорвутся в начале года с места в...
5,2004-01-01,virgo,Сегодняшнее расположение звезд может усилить д...,Дева,Дева. Сегодняшнее расположение звезд может ус...
6,2004-01-01,libra,"В этот день вы, судя по всему, будете в более ...",Весы,"Весы. В этот день вы, судя по всему, будете в..."
7,2004-01-01,scorpio,"Этот день благоприятен для деловых встреч, зна...",Скорпион,Скорпион. Этот день благоприятен для деловых ...
8,2004-01-01,sagittarius,Вероятны ухудшение самочувствия или проблемы с...,Стрелец,Стрелец. Вероятны ухудшение самочувствия или ...
9,2004-01-01,capricorn,"Первый день года будет для вас днем, когда вам...",Козерог,"Козерог. Первый день года будет для вас днем,..."


In [ ]:
df.iloc[1]['horoscope']

' Близнецы. Первый день нового года прекрасно подходит для начинаний, которые вы постоянно откладывали в прошлом году. Отбросьте все сомнения и действуйте. Упор может быть сделан как на повышение профессиональной квалификации, так и на реформацию личной жизни.'

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73477 entries, 0 to 73476
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       73477 non-null  object
 1   sign       73477 non-null  object
 2   text       73477 non-null  object
 3   tr_sign    73477 non-null  object
 4   horoscope  73477 non-null  object
dtypes: object(5)
memory usage: 2.8+ MB


In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
masiv = ''
for i in df['horoscope'].values:
    for g in i:
        masiv += g

In [ ]:
masiv[:300]

' Овен. Любые разногласия во мнениях скоро улягутся, а вы продолжайте делать, как делали, но постарайтесь не наступать на ноги слишком многим, иначе ваши сегодняшние действия сыграют против вас в будущем. Близнецы. Первый день нового года прекрасно подходит для начинаний, которые вы постоянно отклады'

In [ ]:
tok = tokenizer.encode(masiv, add_special_tokens=True)

In [ ]:
tok = np.array(tok)
print(len(tok))
tok[:10]

7128600


array([  415,  4574,    18, 49047, 39588,   567,  1054,  4912,  2493,
         329])

In [ ]:
l = len(tok)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tok[i*l: (i+1)*l])
    else:
        test.extend(tok[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tok), len(train), len(test))

7128600 5702880 1425720


In [ ]:
from transformers import GPT2LMHeadModel


model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
batch_size = 8
max_len = 128
epochs = 10

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)


total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)



def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

5569 1392


In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    2566 MB |    4386 MB |  261304 MB |  258738 MB |\n|       from large pool |    2553 MB |    4365 MB |  259249 MB |  256696 MB |\n|       from small pool |      12 MB |      21 MB |    2054 MB |    2042 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |    2566 MB |    4386 MB |  261304 MB |  258738 MB |\n|       from large pool |    2553 MB |    4365 MB |

In [ ]:
path_full_model = "/content/gdrive/My Drive/Train models/gpt2_fullmodel.pt"

In [ ]:
# готовим тензоры для обучения размера [batch_size, max_len]

def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad()
        loss, logits, _ = model(batch_ids,
                                token_type_ids=None, 
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    if (epoch%1 == 0):
      torch.save(model.state_dict(), path_model)
      torch.save(model, path_full_model)
      print(f'{epoch} save to drive')

    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)

epoch 1/10 : training


  0%|          | 0/5569 [00:00<?, ?it/s]

1 save to drive
epoch 1/10 : validation


  0%|          | 0/1392 [00:00<?, ?it/s]

epoch 2/10 : training


...

  0%|          | 0/1392 [00:00<?, ?it/s]

epoch 10/10 : training


  0%|          | 0/5569 [00:00<?, ?it/s]

10 save to drive
epoch 10/10 : validation


  0%|          | 0/1392 [00:00<?, ?it/s]

In [ ]:
!ls

gdrive	sample_data


In [5]:
from transformers import GPT2LMHeadModel


model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device)
model.load_state_dict(torch.load(path_model))

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

<All keys matched successfully>

In [6]:
import textwrap

In [7]:
prompt = 'Близнецы'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=50,
    num_beams=5,
    do_sample=True,
    temperature=1,
    top_k=50,
    top_p=0.8,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 100), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Близнецы. Сегодня Близнецам не стоит рассчитывать на полную свободу действий. Возможна вынужденная
пауза в делах, вынужденный перерыв в общении с друзьями, коллегами, близкими людьми. Не исключено,
что вы окажетесь перед выбором между двумя направления
------------------
Близнецы. Сегодня Близнецам не стоит рассчитывать на полную свободу действий. Возможны
непредвиденные повороты событий, в том числе в личной жизни. Не исключено, что вы окажетесь
вовлечены в интригу или в небезопасную игру
------------------
Близнецы. Сегодня Близнецам не стоит проявлять инициативу, ввязываться в авантюры и затевать новые
дела. Нежелательно отправляться в путь, приступать к учебе, оформлять документы. Лучше воздержаться
от новых начинаний,
------------------
Близнецы. Сегодня Близнецам не стоит рассчитывать на полную свободу действий, особенно в первой
половине дня. Возможно, вы будете ограничены в чем-то одном, но не исключено, что у вас будет выбор
между несколькими направлениями. В это время
-------

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import gc
gc.collect()

54